# Demography
Пример отчета по расчету социально-демографической структуры аудитории интернет-проектов.


## Описание задачи


Посчитаем аудиторию проекта Kommersant.ru с разбивкой по социально-демографическим переменным:
- Пол
- Пол / Возраст
- Род занятий



Общие параметры:
- Период: Май 2020
- География: Россия 0+
- Население: 12+
- Фильтр по типу использования: нет, считаем по всем (Web Desktop, Web Mobile, App Online, App Offline)

В этом ноутбуке приведен пример расчета Reach Row%, Reach Column%.


Список статистик для расчета:
- Universe (Uni)
- Unweighted Reach (UnwReach)
- Reach
- Reach% (ReachPer)
- Reach Row%
- Reach Column%

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Responsum и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
import logging

from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# pd.set_option("display.max_rows", 200)
# pd.set_option("display.max_colwidth", 50)
# pd.set_option("display.precision", 6)
output_notebook()

## Формируем задания

Необходимо сформировать три задания для Responsum:
- расчет аудитории по проекту Kommersant.ru с разбивкой по соц.-дему
- расчет аудитории Total Internet без разбивки по соц.-дему
- расчет аудитории по проекту Kommersant.ru без разбивки по соц.-дему

### Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# задаем параметры
# выбираем тип установки mobile, т.к. в расчетах участвуют все типы устройств (Web Desktop, Web Mobile, App Online, App Offline)
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop_pre'

# создаем объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)


# задаем период расчета
date_from = '2020-05-01'
date_to = '2020-05-31'

# задаем типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# Проверяем, что значения параметров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

## Социально-демографические переменные
Найдем названия социально-демографических переменных для следущих атрибутов:

- Пол
- Пол / Возраст
- Род занятий

Они необходимы для указания разбивки. Для этого воспользуемся ноутбуком, в котором приведены примеры работы с [каталогами](catalogs.ipynb)

Получили:

- Пол  = SEX
- Пол / Возраст = SEXAGE
- Род занятий = OCCUPATION2

### Получим ID проектов
Для построения отчета нам необходимо получить идентификатор проекта __Kommersant.ru__, для этого воспользуемся ноутбуком, в котором приведены примеры работы с [каталогами](catalogs.ipynb):

- Kommersant.ru  site_id = 118

## Задания
Перейдем к формированию заданий.

### Задание №1. Статистики по проекту Kommersant.ru с разбивкой по соц.-дему

Рассчитаем статистики с разбивкой по проекту и соц.-дему.

In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'kommersant'

# задаем медиа фильтр, в нашем случае это ID проекта Kommersant.ru
media_filter = "site = 118"

# фильтр по демографии отсутствует
demo_filter = None

# задаем список статистик для расчета
statistics = ["UnwReach", "Reach", "ReachPer"]

# указываем разбивки, в нашем случае это разбивка по соц.-дем. переменным:
structure =  {
    "media": ["site"],
    "usetype": False,
    "demo": ["SEX", "SEXAGE", "OCCUPATION2"]
  }

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_kommersant = rtask.result2table(rtask.get_result(task_audience), project_name=project_name, axis_y=['demo'])
df_kommersant

### Дополнительные задания
Также необходимо посчитать статистики Reach Row% и Reach Column% для каждой соц.-дем. группы, для этого сформируем дополнительные задания.

### Задание №2. Статистики по всему Интернету с разбивкой по соц.-дему

Расчет данных по всем проектам займет достаточно много времени у Responsum, поэтому воспользуемся технической страницей WEB-Index, в которой учитывается вся аудитория.
Техническая страница WEB-Index __site_id = 101__

Важно учитывать в случае расчета статистики OTS:
- если считать с ограничением по Технической странице в медиа-фильтре, то полученная статистика OTS будет показывать общее количество заходов в Интернет за указанный период;
- если считать без ограничений в медиа-фильтре (media_filter = None), то полученная статистика OTS будет показывать общее количество загрузок страниц интернет-проектов, содержащихся в вашем медиа-каталоге, за указанный период;
- статистика OTS корректна для работы с десктопными данными и данными по рекламным кампаниям.

In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'total_demo'

# задаем техническую страницу WEB-Index в качестве медиа фильтра
media_filter = 'site = 101'

# фильтр по демографии отсутствует
demo_filter = None

# задаем список статистик для расчета
statistics = ["Uni", "UnwReach", "Reach", "ReachPer"]

# указываем порядок разбивки
structure =  {
    "usetype": False,
    "demo": ["SEX", "SEXAGE", "OCCUPATION2"]
  }

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_total_demo = rtask.result2table(rtask.get_result(task_audience), project_name=project_name, axis_y=['demo'])
df_total_demo

### Задание №3. Статистики по проекту Kommersant.ru без разбивки по соц.-дему

In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'total_project'

# задаем медиа фильтр, в нашем случае это ID проекта Kommersant.ru
media_filter = "(site = 118)"

# фильтр по демографии отсутствует
demo_filter = None

# задаем список статистик для расчета
statistics = ["UnwReach", "Reach", "ReachPer"]

# указываем порядок разбивки
structure =  {
    "usetype": False,
    "media": ["site"]
  }

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_total_prj = rtask.result2table(rtask.get_result(task_audience), project_name=project_name, axis_y=['demo'])
df_total_prj

# Расчет статистик Reach Row% и Reach Column%  

Для этого воспользуемся методом библиотеки Mediascope _calc_row_col_.

Для этого метода на вход передаются три DataFrame:
- данные по проекту/проектам с разбивкой по соц.-дему
- данные по Total Internet с разбивкой по соц.-дему
- данные по проекту/проектам без разбивки по соц.-дему

In [ ]:
df_result = rtask.calc_row_col(df_kommersant, df_total_prj, df_total_demo)
df_result

## Экспорт в Excel

In [ ]:
writer = pd.ExcelWriter(rtask.get_excel_filename('demography-kommersant'))
df_info = rtask.get_report_info()
df_result.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()